### Libraries
Besides [Numpy](https://numpy.org/) and the Pyplot interface to [Matplotlib](https://matplotlib.org/), we will need [OpenCV](https://opencv.org/).

In [71]:
from matplotlib import pyplot as plt

import os

import numpy as np

import cv2 # OpenCV
print (cv2.__version__)

4.10.0


In [72]:
folderpath = './images'

1. Calcolare omografia per ogni immagine e c'è già la funzione di Edi

In [73]:
grid_size = (8,11)
def calculate_homography(image)->np.ndarray:
    return_value, corners = cv2.findChessboardCorners(image, patternSize=grid_size)
    assert(return_value)
    gray = cv2.cvtColor(image.copy(), cv2.COLOR_BGR2GRAY)
    corners = corners.reshape((88,2)).copy()
    criteria = (cv2.TERM_CRITERIA_MAX_ITER | cv2.TERM_CRITERIA_EPS, 100, 0.001)
    cv2.cornerSubPix(gray, corners, (5,5), (-1,-1), criteria)
    A = np.empty((0,9), dtype=float)
    O = np.array([0,0,0]).reshape(1,3)
    square_size = 11 # mm
    for index, corner in enumerate(corners):
        Xpix = corner[0] # in pixel
        Ypix = corner[1]
        grid_size_cv2 = tuple(reversed(grid_size)) # OpenCV and Python store matrices differently
        u_index, v_index = np.unravel_index(index, grid_size_cv2) # convert index from linear to 2D (0-based indexing)
        Xmm = u_index * square_size
        Ymm = v_index * square_size
        m = np.array([Xmm, Ymm, 1]).reshape(1,3)
        A = np.vstack((A, np.hstack((m,O,-Xpix*m))))
        A = np.vstack((A, np.hstack((O,m,-Ypix*m))))

    _, _, Vh = np.linalg.svd(A)
    h = Vh.transpose()[:,-1]
    H = h.reshape(3,3)
    return H

In [74]:
images_path = [os.path.join(folderpath, imagename) for imagename in os.listdir(folderpath) if imagename.endswith(".tiff")]
images_path.sort()

In [75]:
HH = []
for path in images_path:
    image = cv2.imread(path)
    H = calculate_homography(image)
    HH.append(H)

2. per ogni omografia dalla relativa matrice costruire la matrice V + fare funzione che costruisce i vettori v, i, j

In [76]:
def vij_function (H, i, j):
  v=np.zeros(6)
  v[0]=H[0][i]*H[0][j]
  v[1]=H[0][i]*H[1][j]+H[1][i]*H[0][j]
  v[2]=H[1][i]*H[1][j]
  v[3]=H[2][i]*H[0][j]+H[0][i]*H[2][j]
  v[4]=H[2][i]*H[1][j]+H[1][i]*H[2][j]
  v[5]=H[2][i]*H[2][j]
  return v

In [77]:
VV=np.zeros((0,6))

for H in HH:
  v11=vij_function(H, 0, 0)
  v12=vij_function(H, 0, 1)
  v22=vij_function(H, 1, 1)
  V=np.array([v12.T, (v11-v22).T])
  print(V)
  VV=np.vstack((VV, V))
print(VV)

[[-2.13198825e-06  5.63903334e-06  2.09568388e-06  5.12549196e-10
  -9.45018978e-10 -2.77412891e-14]
 [-4.68694420e-06 -8.54923765e-06  6.69908224e-06  1.60720707e-09
   9.79675445e-10 -1.32207328e-13]]
[[-1.06526004e-05 -1.37588766e-05  9.07240259e-06 -4.38326809e-09
   7.63377199e-10 -2.11955853e-13]
 [ 1.20886053e-05 -3.95692999e-05 -1.52584212e-05 -2.36326391e-09
  -8.80077951e-09 -7.26739011e-13]]
[[-2.13944936e-06  2.17732965e-06  3.53923931e-06  1.65752629e-09
  -1.54964443e-09 -2.90579021e-13]
 [-4.41770795e-07 -1.14503640e-05  5.09573919e-06  1.36881802e-09
   4.05900856e-09 -4.70240903e-13]]
[[-1.02637565e-05 -1.21275206e-05  8.90787695e-06 -2.89834687e-09
  -2.98192578e-10 -1.64587134e-13]
 [ 1.09879517e-05 -3.83866705e-05 -1.31421909e-05  1.94842978e-10
  -5.62406050e-09 -1.48689635e-13]]
[[-1.24763542e-05 -3.32936609e-06  1.12964895e-05 -3.16967194e-09
   1.76450123e-10 -1.93520179e-13]
 [ 1.68655121e-06 -4.76110144e-05 -4.82553955e-06 -8.21952511e-10
  -6.13371064e-09 -1.

3. data V trovare B applicando la singular value decomposition

In [81]:
U, Sigma, S_T = np.linalg.svd(VV)
b = S_T.transpose()[:, -1]
B = np.empty((3, 3))
B[0, 0] = b[0]
B[0, 1] = B[1, 0] = b[1]
B[1, 1] = b[2]
B[0, 2] = B[2, 0] = b[3]
B[2, 1] = B[1, 2] = b[4]
B[2, 2] = b[5]
print(B)
eigvals, _ = np.linalg.eigh(B)
print(eigvals)

[[-2.68175698e-07 -4.03041606e-10  1.66921125e-04]
 [-4.03041606e-10 -2.68961779e-07  1.33910848e-04]
 [ 1.66921125e-04  1.33910848e-04 -9.99999977e-01]]
[-1.00000002e+00 -2.68265429e-07 -2.23077259e-07]


4. da B ottenere K e di conseguenza trovare R

In [79]:
L = np.linalg.cholesky(B)
K = np.linalg.inv(L.T) 
# λ = 1/||K−1h1||
K_inv = np.linalg.inv(K)
projections = []

for i, H in enumerate(HH):
    h1 = H[0, :]
    h2 = H[1, :]
    h3 = H[2, :]
    lambdas = 1/np.linalg.norm(K_inv@h1)
    r1 = lambdas * np.linalg.norm(K_inv@h1)
    r2 = lambdas * np.linalg.norm(K_inv@h2)
    t = lambdas * np.linalg.norm(K_inv@h3)
    r3 = np.cross(r1, r2)
    R = np.hstack((r1, r2, r3))
    U, _ , V_t = np.linalg.svd(R)
    R = U@V_t
    P = K@np.hstack(R, t)



LinAlgError: Matrix is not positive definite